# Sample Template

Here is some sample text, and a first block of code

In [ ]:
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'  # always print last expr.
%config InlineBackend.figure_format = 'svg'
%load_ext autoreload
%autoreload 2
%matplotlib inline

import logging

logging.basicConfig(level=logging.INFO)

In [ ]:
import tsdm

In [ ]:
ds = tsdm.datasets.PhysioNet2019()

In [ ]:
ds.metadata

In [ ]:
import pandas as pd
import pyarrow as pa

pa_dict = pa.dictionary(pa.int32(), pa.string())
pd_dict = pd.ArrowDtype(pa_dict)

# create table in pyarrow
data = {"foo": [1, 2], "bar": ["a", "b"]}
schema = {"foo": int, "bar": "category"}

df = pd.DataFrame(data).astype(schema)
df.to_parquet("foo")
df_loaded = pd.read_parquet("foo", dtype_backend="pyarrow")


assert df_loaded.bar.dtype == pd_dict

In [ ]:
df_loaded.bar.dtype == df.bar.dtype

In [ ]:
import pandas as pd
import pyarrow as pa

pa_dict = pa.dictionary(pa.int32(), pa.string())
pd_dict = pd.ArrowDtype(pa_dict)

# create table in pyarrow
data = {"foo": [1, 2], "bar": ["a", "b"]}
schema = pa.schema({"foo": pa.int32(), "bar": pa_dict})
table = pa.table(data, schema=schema)

# serialize in arrow and loading to pandas works
pa.parquet.write_table(table, "demo.parquet")
df = pd.read_parquet("demo.parquet", dtype_backend="pyarrow")

assert df.bar.dtype == pd_dict  # ✔ the dtype is dictionary[int32,string]

# # saving and re-loading doesn't
# df.to_parquet("demo2.parquet")
# pd.read_parquet("demo2.parquet", dtype_backend="pyarrow")

In [ ]:
pa.__version__

In [ ]:
ds.timeseries.hist(figsize=(20, 12), density=True, log=True, bins=20);

In [ ]:
import pandas as pd
import pyarrow as pa

In [ ]:
ts = ds.timeseries.astype({"wd": pd.ArrowDtype(pa.dictionary(pa.int32(), pa.string()))})

In [ ]:
import pandas as pd
import pyarrow as pa

pa_dict = pa.dictionary(pa.int32(), pa.string())
pd_dict = pd.ArrowDtype(pa_dict)

# create table in pyarrow
data = {"foo": [1, 2], "bar": ["a", "b"]}
schema = pa.schema({"foo": pa.int32(), "bar": pa_dict})
table = pa.table(data, schema=schema)

# serliaze in arrow and loading to pandas works
pa.parquet.write_table(table, "demo.parquet")
df = pd.read_parquet("demo.parquet", dtype_backend="pyarrow")

assert df.bar.dtype == pd_dict  # ✔ the dtype is dictionary[int32,string]

# saving and re-loading doesn't
df.to_parquet("demo2.parquet")
pd.read_parquet("demo2.parquet", dtype_backend="pyarrow")